In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from rts.utils import dataframe_from_hdf5
import rts.features.pose as pose
from rts.features.pose import process_video, draw_pose, get_frame, load_all_poses, drop_pose
from rts.features.pose import format_keypoints_to_read, keypoint_name_to_id
from rts.features.pose import KEYPOINTS_NAMES, CONNECTIONS, ANGLES_ASSOCIATIONS


archive_path = "/mnt/g/ioc/data/"
os.path.isdir(archive_path)

sequences_folder = "/mnt/g/ioc/sequences/"
pose_folder = "data/test_poses/"

MODEL = pose.PifPafModel.fast

# Load extracted poses

In [ ]:
poses = load_all_poses(pose_folder, drop_poses=[])
data = dataframe_from_hdf5(archive_path, "metadata")

pose_df = pd.merge(pd.DataFrame(poses), data[["seq_id", "sport"]], left_on="video_name", right_on="seq_id")

# Filter standing poses

In [ ]:
standing_still_angle = {
    "left_elbow": 0.9,
    "right_elbow": 0.9,
    "left_shoulder": 0.15,
    "right_shoulder": 0.15,
    "left_hip": 0.95,
    "right_hip": 0.95,
    "left_knee":0.95,
    "right_knee":0.95,
    "neck":0.5
}

sitting_pose_angle = {
    "left_elbow": 0.6,
    "right_elbow": 0.6,
    "left_shoulder": 0.15,
    "right_shoulder": 0.15,
    "left_hip": 0.5,
    "right_hip": 0.5,
    "left_knee":0.5,
    "right_knee":0.5,
    "neck":0.5
}

In [ ]:
pose_df["drop"] = pose_df.apply(lambda df: drop_pose(df, sitting_pose_angle, drop_threshold=0.2), axis = 1)
pose_df["drop"].value_counts()

In [ ]:
# create a grid of subplots
nrows = 2
ncols = 6
fig, axs = plt.subplots(nrows=2 * nrows, ncols=ncols, figsize=(ncols * 3, 2 * nrows * 3))
axs = axs.flatten()
# iterate over the poses and draw each pose in a subplot
sample_dropped_poses = pose_df[pose_df["drop"]].sample(nrows * ncols)
sample_kept_poses = pose_df[~pose_df["drop"]].sample(nrows * ncols)
sample_poses = pd.concat([sample_dropped_poses, sample_kept_poses]).reset_index(drop=True)
for i, pose in sample_poses.iterrows():
    draw_pose(pose, video_folder=sequences_folder, ax = axs[i], cut = True)
    axs[i].set_title("drop" if pose["drop"] else "keep")
plt.show()

In [ ]:
for sport in pose_df.sport.unique():
    sport_dropped_poses = pose_df[pose_df["drop"] & (pose_df["sport"] == sport)]
    sport_dropped_poses = sport_dropped_poses.sample(np.min([sport_dropped_poses.shape[0], 10])).reset_index(drop=True)
    if sport_dropped_poses.shape[0] == 0:
        continue
    fig, axs = plt.subplots(1, sport_dropped_poses.shape[0], figsize=(sport_dropped_poses.shape[0] * 3, 3))
    if sport_dropped_poses.shape[0] == 1:
        axs = [axs]
    else:
        axs = axs.flatten()

    for i, pose in sport_dropped_poses.iterrows():
        draw_pose(pose, video_folder=sequences_folder, ax = axs[i], cut = True)
        axs[i].set_title(sport)
    plt.show()